# Logging Examples

The following cells shows examples of how to use the various logging modules and functions. It also will display the outputs of each function.

## Install Pre-Requisites and Load RAD Modules

First we need to install the pre-requisites and load the RAD modules to be used in this notebook. The `Init.ps1` script will do this for us.

In [3]:
. .\samples\Init.ps1

Run `Get-Command` to list all the functions available in the RAD modules:

In [4]:
Get-Command -Module Rad-Error-Utils
Get-Command -Module Rad-Text-Utils


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Format-RadErrorMessage                             0.0        Rad-Error-Utils
Function        Get-RadErrorMessage                                0.0        Rad-Error-Utils
Function        Get-RadErrorMessages                               0.0        Rad-Error-Utils
Function        Import-RadErrorsFromYaml                           0.0        Rad-Error-Utils
Function        Invoke-TerminatingCommand                          0.0        Rad-Error-Utils
Function        Set-RadErrorMessages                               0.0        Rad-Error-Utils
Function        Confirm-LoggerIsEnabled                            0.0        Rad-Text-Utils
Function        Show-DebugOutput                                   0.0        Rad-Text-Utils
Function        Show-Error                                         0.0        R

## Enable PoshLog Logger

Run the `Init-Logger.ps1` script to enable the PoshLog logger. 

In [5]:
. .\samples\Init-Logger.ps1

## Rad-Error-Utils

These are examples of how to use the `Rad-Error-Utils` module.

### Get-RadErrorMessages

`Get-RadErrorMessages` returns all the error codes and messages that are available:

In [6]:
Get-RadErrorMessages


Name                           Value
----                           -----
101                            Error 101: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



### Get-RadErrorMessage

`Get-RadErrorMessage` returns a specific error message based on the error code passed in. 

> Some messages allow parameters to be passed in to customize the message. More information on this can be found in the [Format-RadErrorMessage](#format-raderrormessage) section.

In [7]:
Get-RadErrorMessage -ErrorCode 101 -Parameters "param1","param2","param3"

Error 101: Invalid parameter: param1. Expected: param2. Actual: param3.. Error Code: 101


- If a message requires parameters but they are not provided, the function will return an error:

In [8]:
Get-RadErrorMessage -ErrorCode 101 

Exception: Error: The template requires 3 parameters, but only 0 were provided.


Error: Command failed: SubmitCode: Get-RadErrorMessage -ErrorCode 101 

- If the error code is not found, it will return a generic error message:

In [9]:
Get-RadErrorMessage -ErrorCode 200

An unexpected error occurred while executing the command. Error Code: 200


Error: Command failed: SubmitCode: Get-RadErrorMessage -ErrorCode 200

### Format-RadErrorMessage

`Format-RadErrorMessage` is used to format error messages in a consistent way to improve readability and maintainability.

It is also used to insert parameters into a message. It takes in a message and a list of parameters and returns the message with the parameters inserted. The parameters are inserted into the message in the order they are passed in in the list. Parameters are denoted in the message by `{}` and are required if they are present. Not all messages need to have parameters.

In [11]:
$ErrorMessages = Get-RadErrorMessages
$ErrorCode = "500"

Format-RadErrorMessage -Template $ErrorMessages[$ErrorCode] -Parameters "param1","param2"

An unexpected error occurred while executing the command: [param1]. Error message: [param2]


### Set-RadErrorMessages

`Set-RadErrorMessages` allows you to add new error messages. It takes in a hashtable of error codes and messages and adds them to the existing error messages. If the error code already exists, it will overwrite the existing error message.

For comparison purposes, these are the default error error messages available at the start:

In [12]:
# For comparison purposes, these are the default error error messages available

Get-RadErrorMessages


Name                           Value
----                           -----
101                            Error 101: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



1. Now, let's add a few new error messages:

In [13]:
$ErrorMessages = @{
    "1000" = "Unexpected error"
    "1001" = "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    "1004" = "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    "1005" = "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    "1010" = "Unsupported operation - The operation {0} is not supported."
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages


Name                           Value
----                           -----
1000                           Unexpected error
1001                           FileNotFoundError - Failure to locate or access a specified file or…
1004                           PermissionDeniedError - Insufficient permissions to perform the {0}…
1005                           TypeError - Mismatched data types or unsupported operations on a da…
101                            Error 101: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
1010                           Unsupported operation - The operation {0} is not supported.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



2. Next, let's update an existing error message. We should see the message for error code **1000** change from `Unexpected error` to `A RAD error message`:


In [14]:
$ErrorMessages = @{
    "1000" = "A RAD error message"
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages


Name                           Value
----                           -----
1000                           A RAD error message
1001                           FileNotFoundError - Failure to locate or access a specified file or…
1004                           PermissionDeniedError - Insufficient permissions to perform the {0}…
1005                           TypeError - Mismatched data types or unsupported operations on a da…
101                            Error 101: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
1010                           Unsupported operation - The operation {0} is not supported.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



3. There is an *optional* parameter `-Overwrite` that will overwrite all the existing error messages with the new error messages. Let's try that now:

In [15]:
$ErrorMessages = @{
    "500" = "Unexpected error"
    "100" = "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    "101" = "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    "102" = "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    "103" = "Unsupported operation - The operation {0} is not supported."
    "403" = "Authentication error - The user is not authenticated."
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages -Overwrite


Name                           Value
----                           -----
100                            FileNotFoundError - Failure to locate or access a specified file or…
101                            PermissionDeniedError - Insufficient permissions to perform the {0}…
102                            TypeError - Mismatched data types or unsupported operations on a da…
103                            Unsupported operation - The operation {0} is not supported.
403                            Authentication error - The user is not authenticated.
500                            Unexpected error



4. Now when you run Get-RadErrorMessages you will see the only new error messages:

In [16]:
Get-RadErrorMessages


Name                           Value
----                           -----
100                            FileNotFoundError - Failure to locate or access a specified file or…
403                            Authentication error - The user is not authenticated.
101                            PermissionDeniedError - Insufficient permissions to perform the {0}…
500                            Unexpected error
103                            Unsupported operation - The operation {0} is not supported.
102                            TypeError - Mismatched data types or unsupported operations on a da…



### Import-RadErrorsFromYaml

`Import-RadErrorsFromYaml` is used to import error codes and messages from a YAML file. It takes in a path to a YAML file and returns a hashtable of the error codes and messages. This allows you to add your own custom error codes and messages. The YAML file must be in the following format:

```yaml
  # common_errors.yaml
  errors:
    500: "Unexpected error"
    100: "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    101: "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    102: "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    103: "Unsupported operation - The operation {0} is not supported."
    403: "Authentication error - The user is not authenticated."
```



In [18]:
Import-RadErrorsFromYaml -YamlFile ".\samples\errors-example.yaml"

Get-RadErrorMessages


Name                           Value
----                           -----
104                            Docker is not running. Please start Docker and run the script again.
103                            Unsupported operation - The operation {0} is not supported.
500                            Unexpected error
101                            PermissionDeniedError - Insufficient permissions to perform the {0}…
102                            TypeError - Mismatched data types or unsupported operations on a da…
403                            Authentication error - The user is not authenticated.
100                            FileNotFoundError - Failure to locate or access a specified file or…



### Invoke-TerminatingCommand

`Invoke-TerminatingCommand` is used to execute a command and handle any terminating errors that may occur. It takes in a command, an error code and, if necessary, a hashtable of parameters to pass to the command. It will invoke the command and if a terminating error occurs, it will handle the error and return error message, error code and error details using [Show-FatalError](#show-fatalerror). If no terminating error occurs, it will return the output of the command.

In [19]:
Invoke-TerminatingCommand -Command { az account show } -ErrorCode 403

[19:19:05 INF] Executing Command: [ az account show ]
ERROR: Please run 'az login' to setup account.
[19:19:06 FTL] Authentication error - The user is not authenticated.. Error Code: 403
Exception: Authentication error - The user is not authenticated.. Error Code: 403


Error: Command failed: SubmitCode: Invoke-TerminatingCommand -Command { az account sh ...

In [20]:
Invoke-TerminatingCommand -Command { dotnet --version } -ErrorCode 500

[19:19:20 INF] Executing Command: [ dotnet --version ]
[19:19:20 INF] Command executed successfully.
8.0.100


## Rad-Text-Utils

These are examples of how to use the `Rad-Text-Utils` module. When the logger is enabled, these functions will write to a new directory called [~/.utils-logs](~/.utils-logs) and create a new log file `utils-<yyyymmdd>.log`. The logs will write to the console as well as the log file. See below for an example of what a log file will look like:

```text
    04:48:05 [INF] {"ScriptName":"utils"} # THIS IS A TITLE 
    04:48:10 [INF] {"ScriptName":"utils"} This is an example output 
    04:48:13 [DBG] {"ScriptName":"utils"} This is an example debug output 
    04:48:17 [WRN] {"ScriptName":"utils"} This is an example warning 
    04:48:21 [ERR] {"ScriptName":"utils"} This is an example non-terminating error 
    04:48:24 [FTL] {"ScriptName":"utils"} This is an example terminating error 
```

> If a logger is not enabled, the functions will default to use `Write-Host`, `Write-Debug`, or `Write-Warning` accordingly to display in the console.

### Confirm-LoggerIsEnabled

`Confirm-LoggerIsEnabled` is used to confirm that the logger is enabled. It takes in a logger object and returns a boolean value indicating if the logger is enabled or not.

We enabled the logger at the start of this notebook, so this function should return `True`:

In [21]:
Confirm-LoggerIsEnabled

True


## Show-Title

`Show-Title` is used to display a title in the console. It takes in a string and displays it in the console starting with a `#` and capitalizing the text. 

> If you have a logger enabled, the function will use `Write-InfoLog` to display and log the title. Informational logs are displayed in the console with a `[INF]` in front of it.

In [22]:
Show-Title "Getting Azure Resources Without Tag"

[19:21:15 INF] # GETTING AZURE RESOURCES WITHOUT TAG


## Show-Output

`Show-Output` is used to display informational messages to the console. 

> If you have a logger enabled, the function will use `Write-InfoLog` to diplay and log the output. Informational logs are displayed in the console with a `[INF]` in front of it.

In [23]:
Show-Output "Connecting to Azure"

[19:21:22 INF] Connecting to Azure


## Show-DebugOutput

`Show-DebugOutput` is used to display debug messages in the console.

> If you have a logger enabled, the function will use `Write-DebugLog` to log the debug messages. Debug logs are displayed in the log file with a `[DBG]` in front of it.

In [24]:
Show-DebugOutput "This is an example debug output"

## Warnings and Errors

You will use the [Get-RadErrorMessage](#get-raderrormessage) function to get the error messages. The error messages are stored in a hashtable and are accessed by the error code. This message is then passed into the appropriate `Show` function to display the message in the console and log it if a logger is enabled.

## Show-Warning

`Show-Warning` is used to display a warning in the console. It takes in a text string and displays it in the console with a `WARNING:` in front of it. Warnings will not terminate your script and will continue to run.

> If you have a logger enabled, the function will use `Write-WarningLog` to display and log the warning. Warning logs are displayed in the console with a `[WRN]` in front of it.

In [25]:
Show-Warning (Get-RadErrorMessage -ErrorCode 100 -Parameters "file-does-not-exist.ps1")

[19:22:59 WRN] FileNotFoundError - Failure to locate or access a specified file or directory. [file-does-not-exist.ps1]. Error Code: 100


## Show-Error

`Show-Error` is used to display non-terminating errors in the console. It takes in a text string and displays it in the console with a `ERROR:` in front of it.

> If you have a logger enabled, the function will use `Write-ErrorLog` to display and log the error. Error logs are displayed in the console with a `[ERR]` in front of it.

In [26]:
Show-Error (Get-RadErrorMessage -ErrorCode 101 -Parameters "Get-AzResource")

Show-Error: PermissionDeniedError - Insufficient permissions to perform the Get-AzResource operation.. Error Code: 101
[19:23:03 ERR] PermissionDeniedError - Insufficient permissions to perform the Get-AzResource operation.. Error Code: 101


Error: Command failed: SubmitCode: Show-Error (Get-RadErrorMessage -ErrorCode 101 -Pa ...

## Show-FatalError

`Show-FatalError` is used to display a fatal error in the console and terminates the script that is being run. It takes in a text string and displays it in the console with a `FATAL ERROR:` or `Exception` in front of it.

The [Invoke-TerminatingCommand](#invoke-terminatingcommand) function utilizes this function when running a command that may return a terminating error.

> If you have a logger enabled, the function will use `Write-FatalLog` to display and log the fatal error. Fatal logs are displayed in the console with a `[FTL]` in front of it.

In [27]:
Show-FatalError (Get-RadErrorMessage -ErrorCode '104')

[19:23:11 FTL] Docker is not running. Please start Docker and run the script again.. Error Code: 104
Exception: Docker is not running. Please start Docker and run the script again.. Error Code: 104


Error: Command failed: SubmitCode: Show-FatalError (Get-RadErrorMessage -ErrorCode '1 ...

## Next Steps

Now let's run a script that utilizes these functions. This script will import the RAD modules, enable the logger, and then run the various functions in this notebook. We will then look at the log file to see what was logged.

In [30]:
Get-Location

In [31]:
./samples/Show-AzResourcesWithoutTag.ps1

.: /workspaces/rad-powershell-cmdlets/errorhandling/samples/Show-AzResourcesWithoutTag.ps1:25
Line |
  25 |  . .\Init.ps1
     |    ~~~~~~~~~~
     | The term '.\Init.ps1' is not recognized as a name of a cmdlet, function, script file, or
     | executable program. Check the spelling of the name, or if a path was included, verify that
     | the path is correct and try again.
[21:47:58 INF] # GETTING AZURE RESOURCES WITHOUT TAG
.: /workspaces/rad-powershell-cmdlets/errorhandling/samples/Show-AzResourcesWithoutTag.ps1:30
Line |
  30 |  . .\Init-Logger.ps1 -DetailedOutput:$DetailedOutput -SilentMode:$Sile …
     |    ~~~~~~~~~~~~~~~~~
     | The term '.\Init-Logger.ps1' is not recognized as a name of a cmdlet, function, script file,
     | or executable program. Check the spelling of the name, or if a path was included, verify
     | that the path is correct and try again.
[21:47:58 INF] Connecting to Azure
[21:47:58 INF] Executing Command: [ Connect-AzAccount ]
[21:47:58 INF] Command ex

Error: Command failed: SubmitCode: ./samples/Show-AzResourcesWithoutTag.ps1